<a href="https://colab.research.google.com/github/dominiqueGrys/train-musicGen/blob/main/train_musicGen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# os.environ['LC_ALL'] = 'C.UTF-8'
# os.environ['LANG'] = 'C.UTF-8'

In [ ]:
!pip install pydub

In [ ]:
!python3 -m pip install -U git+https://github.com/facebookresearch/audiocraft#egg=audiocraft


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import random
from pydub import AudioSegment

import soundfile as sf
import numpy as np

In [ ]:
# Folder names
input_folder_name = "raw"
output_folder_name = "output"
musicgen_trainer_dir = "musicgen_trainer_dir"

if not os.path.exists(input_folder_name):
    os.makedirs(input_folder_name)
    print(f"'{input_folder_name}' folder created.")

if not os.path.exists(output_folder_name):
    os.makedirs(output_folder_name)
    print(f"'{output_folder_name}' folder created.")

if not os.path.exists(musicgen_trainer_dir):
    os.makedirs(musicgen_trainer_dir)
    print(f"'{musicgen_trainer_dir}' folder created.")

In [ ]:
def delete_files_in_directory(directory_path):
    # Iterate through the files in the directory
    for file_name in os.listdir(directory_path):
        # Construct the full file path
        file_path = os.path.join(directory_path, file_name)

        # Check if it is a file (not a directory)
        if os.path.isfile(file_path):
            # Delete the file
            os.remove(file_path)
            print(f"Deleted: {file_path}")

# Specify the output directory where you want to delete files
output_directory = 'output'

# Delete all files in the output directory
delete_files_in_directory(output_directory)

In [ ]:
import shutil

# Specify the directory you want to delete
directory_path = "/content/musicgen_trainer_dir"

# Delete the directory and all its contents
shutil.rmtree(directory_path)

In [ ]:

def process_audio(file_path, output_dir, segment_length=30):
    # Load audio file
    audio = AudioSegment.from_file(file_path)

    # Get file name without extension for caption
    file_name = os.path.splitext(os.path.basename(file_path))[0]

    # Convert segment length to milliseconds
    segment_length_ms = segment_length * 1000

    # Set the sample rate to 32000 Hz
    audio = audio.set_frame_rate(32000)

    # Calculate the number of segments
    num_segments = (len(audio) + segment_length_ms - 1) // segment_length_ms

    # Loop through segments
    for i in range(num_segments):
        # Get start time for the segment
        start_time = i * segment_length_ms

        # If this is the last segment, adjust start_time
        if i == num_segments - 1:
            start_time = len(audio) - segment_length_ms

        # Get end time for the segment
        end_time = start_time + segment_length_ms

        # Extract the segment
        segment = audio[start_time:end_time]

        # Save the segment
        segment.export(os.path.join(output_dir, f'segment_{i:03d}.wav'), format='wav')

        # Save the caption
        with open(os.path.join(output_dir, f'segment_{i:03d}.txt'), 'w') as f:
            f.write(file_name)

# Directory setup
output_directory = 'output'
samples_directory = 'raw'

# Check if the output directory exists, if not, create it
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Iterate through the files in the "samples" directory
for file_name in os.listdir(samples_directory):
    if file_name.endswith('.wav') or file_name.endswith('.mp3'):
        file_path = os.path.join(samples_directory, file_name)
        process_audio(file_path, output_directory, segment_length=30)


In [ ]:

import librosa

# Directory where the WAV files are saved
output_directory = 'output'

# Iterate through the files in the directory
for file_name in os.listdir(output_directory):
    # Check if the file is a WAV file
    if file_name.endswith('.wav'):
        # Load the audio file
        file_path = os.path.join(output_directory, file_name)
        audio, sample_rate = librosa.load(file_path, sr=None)

        # Check the shape of the audio
        if audio.shape[0] == 32000 * 30:
            print(f"{file_name} has the correct shape: {audio.shape[0]}")
        else:
            print(f"{file_name} does not have the correct shape. Actual shape: {audio.shape[0]}")


#train

In [ ]:
!cd "/content/musicgen_trainer_dir" && git clone https://github.com/chavinlo/musicgen_trainer.git

In [ ]:
!python /content/musicgen_trainer_dir/musicgen_trainer/run.py --dataset_path /content/output

In [ ]:
from audiocraft.models import musicgen
from audiocraft.utils.notebook import display_audio
import torch

In [ ]:
model = musicgen.MusicGen.get_pretrained('small', device='cuda')

model.set_generation_params(duration=8)

model.lm.load_state_dict(torch.load('models/lm_final.pt'))

<All keys matched successfully>

In [ ]:
res = model.generate([
    'country indie jazz, radiohead'
],
    progress=True)
display_audio(res, 32000)